In [29]:
#imports
import time
import numpy as np
import glob
import pickle


from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.callbacks import ModelCheckpoint
from keras import optimizers

In [30]:
np.random.seed(0)

#Declare different variables
#Make variable shape with the width height ansd channels of the image 
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 240, 320, 3
SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)
#Give up your path where all your image files that you want to use for training are located
RAW_PATH = 'D:/S1 2018-2019/ProjectIV/train/'
PATH = RAW_PATH
#In the script for your images you see that they always begin with agg_filez so select all the files that start with this name and call them your train files
TRAIN_DATA_FILES = PATH+'agg_filez*.pkl'
#Make a directory called keras_models inside the map where your images are, here you will give up the path to that file.
MODEL_PATH = RAW_PATH + 'keras_models/'


In [31]:
print(len(TRAIN_DATA_FILES))

46


In [32]:
#to use the images we need to load them
def load_train_data(_train_files):
	#load the data and check the total size of it
	training_data = glob.glob(_train_files)
	t, l = load_file(training_data[0])
	train_size = len(t)*len(training_data)
	print('\nTotal size training data: '+str(train_size))

	# Start array make one for training data and their labels
	total_train = np.zeros([train_size,240,320,3])
	total_label = np.zeros([train_size,3])
	i = 0

	print('Array ready, start loading data')
	for pkl in training_data:	
		print('\n	loading: '+str(pkl))
		t = []
		l = []
		t, l = load_file(pkl)
		for index in range(len(t)):
			total_train[i] = t[index]
			total_label[i] = l[index]
			i += 1

	return total_train, total_label

#load your file
def load_file(_file):
    with open(_file,'rb') as f:  
        t, l = pickle.load(f)
    return t, l



def print_step(_msg,_time):
	print(_msg)
	print_time(_time,time.time())


def print_time(_start,_end):
	time = round((_end - _start),2)
	print((">>> Time take: %s \n")%(time))


In [33]:
print(">> Loading data...")
e0 = time.time()
#load your training data
X, y = load_train_data(TRAIN_DATA_FILES)
print_step((('Data loaded! X shape: %s and y shape: %s')%(X.shape,y.shape)),e0)

# Separate into Train and test
print(">> Separating test and train...")
e1 = time.time()
#plit your data in a train and test set do this for your data and your labels
train, test, train_labels, test_labels = train_test_split(X,y, test_size = 0.2)
print_step((('Train split successful! Train shape: %s and Train_label shape: %s \n')%(train.shape,train_labels.shape)),e1)

print(">> Defining model...")
e2 = time.time()

#Beceaus you don't know which variables will define the best model you can try the different possibilities
learning_rates = [0.001]
#0.0001,0.00001
batch_sizes = [32,64,128]
optz_choices = ['Adam','SGD']
EPOCHS = 25

>> Loading data...

Total size training data: 798
Array ready, start loading data

	loading: D:/S1 2018-2019/ProjectIV/train\agg_filez.pkl

	loading: D:/S1 2018-2019/ProjectIV/train\agg_filez_2.pkl

	loading: D:/S1 2018-2019/ProjectIV/train\agg_filez_color.pkl

	loading: D:/S1 2018-2019/ProjectIV/train\agg_filez_color2.pkl

	loading: D:/S1 2018-2019/ProjectIV/train\agg_filez_flipped.pkl

	loading: D:/S1 2018-2019/ProjectIV/train\agg_filez_flipped_color.pkl

	loading: D:/S1 2018-2019/ProjectIV/train\agg_filez_flipped_color2.pkl
Data loaded! X shape: (798, 240, 320, 3) and y shape: (798, 3)
>>> Time take: 4.83 

>> Separating test and train...
Train split successful! Train shape: (638, 240, 320, 3) and Train_label shape: (638, 3) 

>>> Time take: 0.75 

>> Defining model...


In [ ]:
for lr in learning_rates:
	for bs in batch_sizes:
		for optz in optz_choices:
			layer1 = 100
			layer2 = 50

			log = '\n'
			log += (('\n\n==========\nExecuting for %s layers and %s, with %s epochs, with %s learning rate, with %s batch size, with %s optimizer. Executed at %s')%(layer1,layer2,EPOCHS,lr,bs,optz,e2))

			# Create model
			model = Sequential()
			model.add(Lambda(lambda x: x/127.5-1.0, input_shape=SHAPE))

			model.add(Conv2D(36, (5, 5), activation='elu', strides=(2, 2)))
			model.add(Conv2D(24, (5, 5), activation='elu', strides=(2, 2)))
			model.add(Conv2D(48, (5, 5), activation='elu', strides=(2, 2)))
			model.add(Conv2D(64, (3, 3), activation='elu'))
			model.add(Conv2D(64, (3, 3), activation='elu'))

			model.add(Dropout(0.5))
			model.add(Flatten())

			model.add(Dense(layer1, input_dim=SHAPE, activation='elu'))
			model.add(Dense(layer2, activation='elu'))
			model.add(Dense(3))

			print(model.summary())
			print_step("Model defined!",e2)

			print(">> Compiling model...")
			e3 = time.time()

			# Compile model
			learn_rate = lr
			BATCH_SIZE = bs
			if (optz == 'SGD'):
				opt = optimizers.SGD(lr=learn_rate)
			else:
				opt = optimizers.Adam(lr=learn_rate)

			model.compile(loss='mean_squared_error', optimizer=opt, metrics=['accuracy'])

			checkpoint = ModelCheckpoint('model-{epoch:03d}.h5',
	                             monitor='val_loss',
	                             verbose=0,
	                             save_best_only=True,
	                             mode='auto')

			print_step("Model compiled!", e3)

			print(">> Fiting model...")
			e4 = time.time()
			# Fit the model
			model.fit(train
					, train_labels
					, validation_data=(test, test_labels)
					, epochs=EPOCHS
					, batch_size=BATCH_SIZE
					, verbose=1)
			log += (('\n 	Train time: %s')%(time.time() - e4))
			print_step("Fit finished!", e4)

			print(">> Evaluating model...")
			e5 = time.time()

			# Evaluate train data
			scores_train = model.evaluate(train, train_labels)
			eval_train = (("\n 	Evaluate Train: %s: %.2f%%") % (model.metrics_names[1], scores_train[1]*100))
			log += '\n'+eval_train
			print(eval_train)

			# Evaluate test data
			scores_test = model.evaluate(test, test_labels)
			eval_test = (("\n 	Evaluate Test: %s: %.2f%%") % (model.metrics_names[1], scores_test[1]*100))
			log += '\n'+eval_test
			print(eval_test)

			log += (('\n\n 	Total time: %s')%(time.time() - e2))

			print_step("Evaluation finished!", e5)

			print(">> Saving model...")
			e6 = time.time()

			# save to json
			name = MODEL_PATH+'keras_model_'+str(lr)+'_learning_rate_'+str(bs)+'_batch_size_'+(optz)+'_optimizer_'+str(round(scores_test[1]*100))+'_acc'
			log += (('\n\n 	saved with name "%s"')%(name))
			model.save(name+".model")

			with open(MODEL_PATH+'models_log.txt','a') as log_file:
				log_file.write(log)

			print_step("Model saved!", e6)

print_step("Finished!!", e0)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 240, 320, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 118, 158, 36)      2736      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 77, 24)        21624     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 37, 48)        28848     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 25, 35, 64)        27712     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 23, 33, 64)        36928     
_________________________________________________________________
dropout_1 (Dropout)          (None, 23, 33, 64)        0         
__________